In [1]:
pip install --upgrade yfinance

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
bs = yf.Ticker("AZN").quarterly_balance_sheet

#no seasonality is assumed for biotech
terminal_growth = 0.01
federal_rate = 1.0428
market_growth = 1.113
DKK_TO_USD = 0.14
JPY_TO_USD = 0.0066

In [3]:
ticker_list = ["AZN", "LLY", "NVO", "TAK", "BMY"] #, "PFE", "GSK", "SNY", "ROG.SW", "TAK", "NVO", "NVS", "GILD"

beta_dict = {}
for ticker in ticker_list:
    try:
        ticker_info = yf.Ticker(ticker).info
        beta = ticker_info.get("beta")
        if beta is not None:
            beta_dict[ticker] = beta
        else:
            print(f"Beta information not available for {ticker}")
    except Exception as e:
        print(f"An error occurred while fetching data for {ticker}: {e}")

beta_df = pd.DataFrame(list(beta_dict.items()), columns=['Ticker', 'Beta'])
beta_df.set_index('Ticker', inplace=True)
beta_df


,Beta
Ticker,
AZN,0.201
LLY,0.412
NVO,0.192
TAK,0.456
BMY,0.451


In [4]:
# prompt: make a dataframe with Close prices for all companies in ticker_list for the 5 most recent dates in columns of bs dataframe - use the 5 recent most dates that appear as columns in bs - if the date is not in the prices data, pick the closest available date to it - only put the prices in the dataframe

# Get the 5 most recent dates from the columns of the 'bs' DataFrame.
dates = bs.columns[:5]

# Create an empty dictionary to store the prices for each company.
prices_dict = {}

# Loop through each ticker in the ticker_list.
for ticker in ticker_list:
    # Download the historical data for the current ticker.
    data = yf.download(ticker, period="max")

    # Create an empty list to store the prices for the current ticker.
    prices = []

    # Loop through each date in the 'dates' list.
    for date in dates:
        # Convert the date to a datetime object.
        date_dt = pd.to_datetime(date)

        # Find the closest available date in the historical data.
        closest_date = min(data.index, key=lambda x: abs(x - date_dt))

        # Get the closing price for the closest date.
        price = data.loc[closest_date, 'Close'].values[0]

        # Append the price to the 'prices' list.
        prices.append(price)

    # Store the prices for the current ticker in the 'prices_dict'.
    prices_dict[ticker] = prices

# Create a DataFrame from the 'prices_dict'.
prices_df = pd.DataFrame(prices_dict, index=dates)

# Display the DataFrame.
prices_df


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,AZN,LLY,NVO,TAK,BMY
2024-12-31,64.597305,770.671753,86.019997,13.24,55.942513
2024-09-30,76.812828,882.953430,119.070000,14.22,50.608620
2024-06-30,76.372993,910.015991,144.877731,13.00,39.791859
2024-03-31,65.897919,755.640442,127.074371,13.67,50.467922
2023-12-31,64.997192,578.143738,102.325958,14.27,48.307255


In [9]:
balance_sheet_map = {
    'Ordinary Shares Number':'Ordinary Shares Number',
    'Cash Cash Equivalents And Short Term Investments':'Cash & Short Term Investments',
    'Accounts Receivable':'Accounts Receivable',
    'Inventory':'Inventory',
    'Current Assets':'Current Assets',
    'Total Assets':'Assets',
    'Total Liabilities Net Minority Interest':'Total Liabilities',
    'Accounts Payable':'Accounts Payable',
    'Long Term Debt And Capital Lease Obligation':'Long Term Debt And Capital Lease Obligation',
    'Current Debt And Capital Lease Obligation':'Current Debt And Capital Lease Obligation',
    'Current Liabilities':'Total Current Liabilities',
    'Stockholders Equity':'Total Equity'
}

income_statement_map = {
    'Total Revenue':'Revenue',
    'Cost Of Revenue':'Cost of Revenue',
    'Gross Profit':'Gross Profit',
    'EBITDA':'EBITDA',
    'EBIT':'EBIT',
    'Interest Expense':'Interest Expense',
    'Net Income':'Net Income',
    'Net Income Continuous Operations':'Net Income Continuous Operations',
    'Basic EPS':'Basic EPS',
    'Tax Rate For Calcs':'Tax Rate For Calcs',
    #'Basic Average Shares':'Basic Average Shares'
}

cash_flow_map = {
    'Operating Cash Flow':'Operating Cash Flow',
    'Free Cash Flow':'Free Cash Flow',
    'Cash Dividends Paid': 'Cash Dividends Paid'
}

In [20]:
fin_all = pd.DataFrame()

for ticker in ticker_list:
  print(ticker)

  multiplier = 1
  if ticker == 'TAK':
    multiplier = JPY_TO_USD
    terminal_growth = 0.005
  if ticker == 'NVO':
    multiplier = DKK_TO_USD

  bs = yf.Ticker(ticker).quarterly_balance_sheet
  bs = bs[bs.index.isin(balance_sheet_map.keys())]
  bs = bs.sort_index(axis=1, ascending=False)
  #multiple all number fields with multiplier in bs
  for idx in list(balance_sheet_map.keys())[1:]:
    bs.loc[idx,:] = bs.loc[idx,:] * multiplier

  income = yf.Ticker(ticker).quarterly_income_stmt
  income = income[income.index.isin(income_statement_map.keys())]
  income = income.sort_index(axis=1, ascending=False)
  #multiple all number keys with multiplier in income
  for idx in list(income_statement_map.keys())[:-1]:
    income.loc[idx,:] = income.loc[idx,:] * multiplier

  cf = yf.Ticker(ticker).quarterly_cash_flow
  cf = cf[cf.index.isin(cash_flow_map.keys())]
  cf = cf.sort_index(axis=1, ascending=False)
  for idx in list(cash_flow_map.keys()):
    cf.loc[idx,:] = cf.loc[idx,:] * multiplier

  fin = pd.concat([bs, cf, income], axis=0)
  fin = fin.sort_index(axis=1, ascending=False)
  fin.loc['Close price', :] = prices_df.loc[:, ticker]

  fin = fin.drop(columns=[col for col in fin if col not in prices_df.index])

  fin = fin.T
  fin['Market cap'] = fin['Close price'] * fin['Ordinary Shares Number']
  fin['Debt to equity'] = fin['Total Liabilities Net Minority Interest'] / fin['Stockholders Equity']
  fin['Interest coverage'] = fin['EBITDA'] / fin['Interest Expense']
  fin['Current ratio'] = fin['Current Assets'] / fin['Current Liabilities']
  fin['Cash ratio'] = fin['Cash Cash Equivalents And Short Term Investments'] / fin['Current Liabilities']
  fin['Sales to assets'] = fin['Total Revenue'] / fin['Total Assets']
  fin['Days in inventory'] = 365 * fin['Inventory'] / fin['Cost Of Revenue']
  fin['Account receiveable days'] = 365 * fin['Accounts Receivable'] / fin['Total Revenue']
  fin['Account payable days'] = 365 * fin['Accounts Payable'] / fin['Cost Of Revenue']
  fin['Gross profit margin'] = fin['Gross Profit'] / fin['Total Revenue']
  fin['EBITDA margin'] = fin['EBITDA'] / fin['Total Revenue']
  fin['EBIT margin'] = fin['EBIT'] / fin['Total Revenue']
  fin['Profit margin from continuing operations'] = fin['Net Income Continuous Operations'] / fin['Total Revenue']
  fin['Net profit margin'] = fin['Net Income'] / fin['Total Revenue']
  fin['Return on Assets'] = fin['Net Income'] / fin['Total Revenue']
  fin['Cash Dividends Paid'] = np.abs(fin['Cash Dividends Paid'])
  fin['Dividend per share'] = fin['Cash Dividends Paid'] / fin['Ordinary Shares Number']
  fin['Payout ratio'] = fin['Cash Dividends Paid'] / fin['Net Income']
  fin['Return on Equity'] = fin['Net Income'] / fin['Stockholders Equity']
  fin['Basic EPS'] = np.where(fin['Basic EPS'].isna(), fin['Net Income'] / fin['Ordinary Shares Number'], fin['Basic EPS'])
  fin['P/E'] = fin['Close price'] / fin['Basic EPS']
  fin['Dividend yield'] = fin['Dividend per share'] / fin['Close price']
  fin['Beta'] = beta_df.loc[ticker].values[0]
  revenue_gr = (fin['Total Revenue'].diff(-1).shift(1) / fin['Total Revenue']).mean()
  if (fin['Cash Dividends Paid']>0).all():
    dividend_gr = (fin['Dividend per share'].diff(-1).shift(1) / fin['Dividend per share']).mean()
  else:
    dividend_gr = 0
  fcf_gr = (fin['Free Cash Flow'].diff(-1).shift(1) / fin['Free Cash Flow']).mean()
  fin['RevGR'] = fin['Total Revenue'].diff(-1).shift(1) / fin['Total Revenue']
  fin['RevGR'] = fin['RevGR'].fillna(revenue_gr)
  if (fin['Cash Dividends Paid']>0).all():
    fin['DivGR'] = fin['Dividend per share'].diff(-1).shift(1) / fin['Dividend per share']
  else:
    fin['DivGR'] = 0
  fin['DivGR'] = fin['DivGR'].fillna(dividend_gr)
  fin['FCFGR'] = fin['Free Cash Flow'].diff(-1).shift(1) / fin['Free Cash Flow']
  fin['FCFGR'] = fin['FCFGR'].fillna(fcf_gr)
  #convert to yearly
  fin['RevGR'] = fin['RevGR'] * 4
  fin['DivGR'] = fin['DivGR'] * 4
  fin['FCFGR'] = fin['FCFGR'] * 4
  fin['rE(Beta)'] = federal_rate + fin['Beta']*(market_growth-federal_rate)-1 #CAPM
  fin['rE(Dividend)'] = fin['Dividend per share']*(1+fin['DivGR'])/fin['Close price'] + fin['DivGR']
  fin['rE'] = np.where((fin['rE(Dividend)']>0.03) & (fin['rE(Dividend)']<0.15), fin['rE(Dividend)'], fin['rE(Beta)'])
  fin['Total debt'] = fin['Current Debt And Capital Lease Obligation'] + fin['Long Term Debt And Capital Lease Obligation']
  fin['rD'] = fin['Interest Expense'] / fin['Total debt']
  #getting rid of negatives
  fin['rD'] = np.where(fin['rD'] > 0, fin['rD'], fin['rD'].shift(-1))
  fin['rD'] = np.where(fin['rD'].isna(), fin['rD'].shift(1), fin['rD'])
  fin['Free Cash Flow'] = np.where(fin['Free Cash Flow'] > 0, fin['Free Cash Flow'], fin['Free Cash Flow'].shift(-1))
  fin['Free Cash Flow'] = np.where(fin['Free Cash Flow'].isnull(), fin['Free Cash Flow'].shift(1), fin['Free Cash Flow'])
  fin['wE'] = fin['Market cap'] / (fin['Market cap'] + fin['Total debt'])
  fin['wD'] = 1 - fin['wE']
  fin['WACC'] = fin['wE'] * fin['rE'] + fin['wD'] * fin['rD'] * (1-fin['Tax Rate For Calcs'])
  fin['Growth'] = np.where( (fin['RevGR'] > terminal_growth) & (fin['FCFGR'] > terminal_growth), np.where(fin['RevGR'] > fin['FCFGR'], fin['FCFGR'], fin['RevGR']), np.where(fin['RevGR'] > terminal_growth, fin['RevGR'], terminal_growth) )
  fin['RollingFCF'] = fin['Free Cash Flow'][::-1].rolling(2).mean()[::-1].fillna(fin['Free Cash Flow'])
  fin['FCF6'] = fin['RollingFCF'] * 4 * (1+fin['Growth'])**6 #*4 to convert to yearly
  fin['Discounted terminal value'] = (fin['FCF6'] / (fin['WACC']-terminal_growth)) / (1+fin['WACC'])**5
  fin['Growing annuity'] = fin['RollingFCF']*(1+fin['Growth'])/(fin['WACC']-fin['Growth']) * (1-((1+fin['Growth'])/(1+fin['WACC']))**5)
  fin['Total value now'] = fin['Discounted terminal value'] + fin['Growing annuity']
  fin['DCF fair share value'] = fin['Total value now'] / fin['Ordinary Shares Number']
  fin['Share px relative to fair value'] = fin['Close price'] / fin['DCF fair share value']

  fin['ticker'] = ticker
  fin = fin.reset_index()
  fin.rename(columns={'index': 'Date'}, inplace=True)

  fin_all = pd.concat([fin_all, fin], axis=0)
fin = fin_all.set_index('ticker')

AZN
LLY


<ipython-input-20-7921cfeead7b>:70: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fin['RevGR'] = fin['RevGR'].fillna(revenue_gr)
<ipython-input-20-7921cfeead7b>:75: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fin['DivGR'] = fin['DivGR'].fillna(dividend_gr)
<ipython-input-20-7921cfeead7b>:77: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
 

NVO
TAK


<ipython-input-20-7921cfeead7b>:70: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fin['RevGR'] = fin['RevGR'].fillna(revenue_gr)
<ipython-input-20-7921cfeead7b>:77: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fin['FCFGR'] = fin['FCFGR'].fillna(fcf_gr)
<ipython-input-20-7921cfeead7b>:96: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fin[

BMY


<ipython-input-20-7921cfeead7b>:70: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fin['RevGR'] = fin['RevGR'].fillna(revenue_gr)
<ipython-input-20-7921cfeead7b>:75: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fin['DivGR'] = fin['DivGR'].fillna(dividend_gr)
<ipython-input-20-7921cfeead7b>:77: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
 

In [23]:
fin_all.to_csv('fin_all.csv')

In [22]:
#sanity check
fin_all.reset_index()[['ticker', 'Date']].value_counts().sort_values()

ticker  Date      
AZN     2023-12-31    1
        2024-09-30    1
        2024-12-31    1
BMY     2023-12-31    1
        2024-03-31    1
        2024-06-30    1
        2024-09-30    1
        2024-12-31    1
LLY     2023-12-31    1
        2024-03-31    1
AZN     2024-03-31    1
        2024-06-30    1
LLY     2024-06-30    1
NVO     2023-12-31    1
        2024-03-31    1
        2024-06-30    1
        2024-09-30    1
        2024-12-31    1
TAK     2023-12-31    1
        2024-03-31    1
        2024-06-30    1
        2024-09-30    1
LLY     2024-09-30    1
        2024-12-31    1
TAK     2024-12-31    1
Name: count, dtype: int64

In [17]:
fin_all = pd.DataFrame()

for ticker in ['TAK']:

  multiplier = 1
  if ticker == 'TAK':
    multiplier = JPY_TO_USD
    terminal_growth = 0.005
  if ticker == 'NVO':
    multiplier = DKK_TO_USD

  bs = yf.Ticker(ticker).quarterly_balance_sheet
  bs = bs[bs.index.isin(balance_sheet_map.keys())]
  bs = bs.sort_index(axis=1, ascending=False)
  #multiple all number fields with multiplier in bs
  for idx in list(balance_sheet_map.keys())[1:]:
    bs.loc[idx,:] = bs.loc[idx,:] * multiplier

  income = yf.Ticker(ticker).quarterly_income_stmt
  income = income[income.index.isin(income_statement_map.keys())]
  income = income.sort_index(axis=1, ascending=False)
  #multiple all number keys with multiplier in income
  for idx in list(income_statement_map.keys())[:-1]:
    income.loc[idx,:] = income.loc[idx,:] * multiplier

  cf = yf.Ticker(ticker).quarterly_cash_flow
  cf = cf[cf.index.isin(cash_flow_map.keys())]
  cf = cf.sort_index(axis=1, ascending=False)
  for idx in list(cash_flow_map.keys()):
    cf.loc[idx,:] = cf.loc[idx,:] * multiplier

  fin = pd.concat([bs, cf, income], axis=0)
  fin = fin.sort_index(axis=1, ascending=False)
  fin.loc['Close price', :] = prices_df.loc[:, ticker]

  fin = fin.drop(columns=[col for col in fin if col not in prices_df.index])

  fin = fin.T
  fin['Market cap'] = fin['Close price'] * fin['Ordinary Shares Number']
  fin['Debt to equity'] = fin['Total Liabilities Net Minority Interest'] / fin['Stockholders Equity']
  fin['Interest coverage'] = fin['EBITDA'] / fin['Interest Expense']
  fin['Current ratio'] = fin['Current Assets'] / fin['Current Liabilities']
  fin['Cash ratio'] = fin['Cash Cash Equivalents And Short Term Investments'] / fin['Current Liabilities']
  fin['Sales to assets'] = fin['Total Revenue'] / fin['Total Assets']
  fin['Days in inventory'] = 365 * fin['Inventory'] / fin['Cost Of Revenue']
  fin['Account receiveable days'] = 365 * fin['Accounts Receivable'] / fin['Total Revenue']
  fin['Account payable days'] = 365 * fin['Accounts Payable'] / fin['Cost Of Revenue']
  fin['Gross profit margin'] = fin['Gross Profit'] / fin['Total Revenue']
  fin['EBITDA margin'] = fin['EBITDA'] / fin['Total Revenue']
  fin['EBIT margin'] = fin['EBIT'] / fin['Total Revenue']
  fin['Profit margin from continuing operations'] = fin['Net Income Continuous Operations'] / fin['Total Revenue']
  fin['Net profit margin'] = fin['Net Income'] / fin['Total Revenue']
  fin['Return on Assets'] = fin['Net Income'] / fin['Total Revenue']
  fin['Cash Dividends Paid'] = np.abs(fin['Cash Dividends Paid'])
  fin['Dividend per share'] = fin['Cash Dividends Paid'] / fin['Ordinary Shares Number']
  fin['Payout ratio'] = fin['Cash Dividends Paid'] / fin['Net Income']
  fin['Return on Equity'] = fin['Net Income'] / fin['Stockholders Equity']
  fin['Basic EPS'] = np.where(fin['Basic EPS'].isna(), fin['Net Income'] / fin['Ordinary Shares Number'], fin['Basic EPS'])
  fin['P/E'] = fin['Close price'] / fin['Basic EPS']
  fin['Dividend yield'] = fin['Dividend per share'] / fin['Close price']
  fin['Beta'] = beta_df.loc[ticker].values[0]
  revenue_gr = (fin['Total Revenue'].diff(-1).shift(1) / fin['Total Revenue']).mean()
  if (fin['Cash Dividends Paid']>0).all():
    dividend_gr = (fin['Dividend per share'].diff(-1).shift(1) / fin['Dividend per share']).mean()
  else:
    dividend_gr = 0
  fcf_gr = (fin['Free Cash Flow'].diff(-1).shift(1) / fin['Free Cash Flow']).mean()
  fin['RevGR'] = fin['Total Revenue'].diff(-1).shift(1) / fin['Total Revenue']
  fin['RevGR'] = fin['RevGR'].fillna(revenue_gr)
  if (fin['Cash Dividends Paid']>0).all():
    fin['DivGR'] = fin['Dividend per share'].diff(-1).shift(1) / fin['Dividend per share']
  else:
    fin['DivGR'] = 0
  fin['DivGR'] = fin['DivGR'].fillna(dividend_gr)
  fin['FCFGR'] = fin['Free Cash Flow'].diff(-1).shift(1) / fin['Free Cash Flow']
  fin['FCFGR'] = fin['FCFGR'].fillna(fcf_gr)
  #convert to yearly
  fin['RevGR'] = fin['RevGR'] * 4
  fin['DivGR'] = fin['DivGR'] * 4
  fin['FCFGR'] = fin['FCFGR'] * 4
  fin['rE(Beta)'] = federal_rate + fin['Beta']*(market_growth-federal_rate)-1 #CAPM
  fin['rE(Dividend)'] = fin['Dividend per share']*(1+fin['DivGR'])/fin['Close price'] + fin['DivGR']
  fin['rE'] = np.where((fin['rE(Dividend)']>0.03) & (fin['rE(Dividend)']<0.15), fin['rE(Dividend)'], fin['rE(Beta)'])
  fin['Total debt'] = fin['Current Debt And Capital Lease Obligation'] + fin['Long Term Debt And Capital Lease Obligation']
  fin['rD'] = fin['Interest Expense'] / fin['Total debt']
  #getting rid of negatives
  fin['rD'] = np.where(fin['rD'] > 0, fin['rD'], fin['rD'].shift(-1))
  fin['rD'] = np.where(fin['rD'].isna(), fin['rD'].shift(1), fin['rD'])
  fin['Free Cash Flow'] = np.where(fin['Free Cash Flow'] > 0, fin['Free Cash Flow'], fin['Free Cash Flow'].shift(-1))
  fin['Free Cash Flow'] = np.where(fin['Free Cash Flow'].isnull(), fin['Free Cash Flow'].shift(1), fin['Free Cash Flow'])
  fin['wE'] = fin['Market cap'] / (fin['Market cap'] + fin['Total debt'])
  fin['wD'] = 1 - fin['wE']
  fin['WACC'] = fin['wE'] * fin['rE'] + fin['wD'] * fin['rD'] * (1-fin['Tax Rate For Calcs'])
  fin['Growth'] = np.where( (fin['RevGR'] > terminal_growth) & (fin['FCFGR'] > terminal_growth), np.where(fin['RevGR'] > fin['FCFGR'], fin['FCFGR'], fin['RevGR']), np.where(fin['RevGR'] > terminal_growth, fin['RevGR'], terminal_growth) )
  fin['RollingFCF'] = fin['Free Cash Flow'][::-1].rolling(2).mean()[::-1].fillna(fin['Free Cash Flow'])
  fin['FCF6'] = fin['RollingFCF'] * 4 * (1+fin['Growth'])**6 #*4 to convert to yearly
  fin['Discounted terminal value'] = (fin['FCF6'] / (fin['WACC']-terminal_growth)) / (1+fin['WACC'])**5
  fin['Growing annuity'] = fin['RollingFCF']*(1+fin['Growth'])/(fin['WACC']-fin['Growth']) * (1-((1+fin['Growth'])/(1+fin['WACC']))**5)
  fin['Total value now'] = fin['Discounted terminal value'] + fin['Growing annuity']
  fin['DCF fair share value'] = fin['Total value now'] / fin['Ordinary Shares Number']
  fin['Share px relative to fair value'] = fin['Close price'] / fin['DCF fair share value']

  fin['ticker'] = ticker
  fin = fin.reset_index()
  fin.rename(columns={'index': 'Date'}, inplace=True)

  fin_all = pd.concat([fin_all, fin], axis=0)

<ipython-input-17-bc74cfc315a9>:69: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fin['RevGR'] = fin['RevGR'].fillna(revenue_gr)
<ipython-input-17-bc74cfc315a9>:74: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fin['DivGR'] = fin['DivGR'].fillna(dividend_gr)
<ipython-input-17-bc74cfc315a9>:76: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
 

In [ ]:
# prompt: the values didn't change in the table after multiplying them by multiplier - what is the issue?

import yfinance as yf
import pandas as pd
import numpy as np

# ... (rest of your code)

  bs = yf.Ticker(ticker).quarterly_balance_sheet
  bs = bs[bs.index.isin(balance_sheet_map.keys())]
  bs = bs.sort_index(axis=1, ascending=False)
  #multiple all number fields with multiplier in bs
  # The issue was with how you were selecting the columns to multiply
  # You were using the index names of the *remapped* balance sheet, but applying it to the *original* index names
  # This is fixed by using .loc to select based on the original index and then multiplying
  for col in bs.columns:
    bs[col] = bs.loc[list(balance_sheet_map.keys())[1:-1], col] * multiplier

  income = yf.Ticker(ticker).quarterly_income_stmt
  # ... (rest of your code)


In [11]:
fin = fin.set_index('Date')

In [12]:
fin[['Free Cash Flow', 'rE', 'rD', 'WACC', 'Interest Expense', 'wE', 'wD', 'RevGR', 'FCFGR', 'Growth', 'Close price', 'DCF fair share value', 'Share px relative to fair value']]

,Free Cash Flow,rE,rD,WACC,Interest Expense,wE,wD,RevGR,FCFGR,Growth,Close price,DCF fair share value,Share px relative to fair value
Date,,,,,,,,,,,,,
2024-12-31,2158893000.0,0.074811,0.006528,0.033218,208533600.0,0.396499,0.603501,0.041646,6.776783,0.041646,13.24,170.562764,0.077625
2024-09-30,1244172600.0,0.074811,0.013552,0.036417,451796400.0,0.403393,0.596607,-0.108548,2.940815,0.005000,14.22,52.57537,0.270469
2024-06-30,214539600.0,0.074811,0.010891,0.031832,393960600.0,0.360565,0.639435,-0.105802,19.197071,0.005000,13.0,70.555743,0.184251
2024-03-31,1414129200.0,0.074811,0.014272,0.036599,-350644800.0,0.401527,0.598473,0.598061,-3.393154,0.598061,13.67,1063.779569,0.01285
2023-12-31,457558200.0,0.074811,0.014272,0.037846,439348800.0,0.421056,0.578944,-0.217127,8.362398,0.005000,14.27,31.717904,0.449904
